In [ ]:
import pandas as pd
import numpy as np
from PIL import Image

In [ ]:
import torch
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# import the data
from google.colab import drive
drive.mount('/content/drive')

import os
image_dir = '/content/drive/MyDrive/6.S058 DR Diagnosis/images' # Replace with your directory path
# image_files = os.listdir(image_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Preprocess the data to be the same as the ImageNet ones
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Resize directly because images are square
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    )
])

In [ ]:
# Load the dataset using ImageFolder
image_dataset = datasets.ImageFolder(root=image_dir, transform=transform)

# Create a data loader to iterate through the dataset in batches
data_loader = DataLoader(image_dataset, batch_size=32, shuffle=True)

# The labels are automatically assigned based on the folder names
print(image_dataset.class_to_idx)

# Iterate through the data loader
for images, labels in data_loader:
    # Process the images and labels
    print(images.shape, labels.shape)
    break

{'class_0': 0, 'class_1': 1, 'class_2': 2, 'class_3': 3, 'class_4': 4, 'class_nan': 5}
torch.Size([32, 3, 224, 224]) torch.Size([32])


In [ ]:
# Load pre-trained model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = models.resnet18(pretrained=True)
model = model.eval().to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 186MB/s]


In [ ]:
# Run ResNet18 on the data without any further training

# Inference mode
all_outputs = []

with torch.no_grad():  # very important! disables gradients (faster, no memory waste)
    for inputs, _ in data_loader:  # usually dataloader returns (input, label) tuples
        inputs = inputs.to(device)
        outputs = model(inputs)  # forward pass batch through model
        all_outputs.append(outputs.cpu())  # collect outputs (move to CPU to save GPU memory)

# (optional) concatenate all outputs
all_outputs = torch.cat(all_outputs, dim=0)

In [ ]:
# get labels
true_labels = torch.tensor(image_dataset.targets)

# get the highest confidence class
predicted_labels = all_outputs.argmax(dim=1)
correct = (predicted_labels == true_labels).sum().item()
total = true_labels.size(0)
accuracy = correct / total

print(f"Accuracy: {accuracy:.4f}")